In [1]:
from json import JSONDecodeError

from utils import *
from output_formatter import transform_to_prodigy
from nervaluate import Evaluator

In [2]:
model = get_base_model()
tokenizer = get_base_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
sentences = []
true_responses = []
pred_responses = []

In [4]:
with open('data/CDR_TestSet.json', 'r') as file:
    data = json.load(file)

for item in data:
    if 'user' not in item or 'assistant' not in item:
        print("problem with item:", item)
    sentences.append(item['user'])
    true_responses.append(transform_to_prodigy(item['user'], item['assistant']))

In [5]:
generated_responses = []
for sentence in sentences:
    generated_responses.append(generate_response(sentence, model, tokenizer))

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [20]:
invalid_response_count = 0
for i in range(len(generated_responses)):
    try:
        transform_to_prodigy(sentences[i], generated_responses[i])
    except:
        invalid_response_count += 1

print(f"Invalid json format: {invalid_response_count} out of {len(generated_responses)} times")

Invalid json format: 345 out of 2422 times


In [15]:
with open('output/base_model.txt', 'w') as file:
    file.write("\n\n".join(generated_responses))

In [36]:
pred_responses = []
for i in range(len(sentences)):
    predicted = []
    try:
        predicted = transform_to_prodigy(sentences[i], generated_responses[i])
    except:
        pass
    pred_responses.append(predicted)

In [37]:
evaluator = Evaluator(true_responses, pred_responses, tags=['Disease', 'Chemical'])
results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
results

{'ent_type': {'correct': 5837,
  'incorrect': 225,
  'partial': 0,
  'missed': 3758,
  'spurious': 3054,
  'possible': 9820,
  'actual': 9116,
  'precision': 0.6403027643703378,
  'recall': 0.5943991853360489,
  'f1': 0.6164976763836079},
 'partial': {'correct': 5009,
  'incorrect': 0,
  'partial': 1053,
  'missed': 3758,
  'spurious': 3054,
  'possible': 9820,
  'actual': 9116,
  'precision': 0.6072290478279947,
  'recall': 0.5636965376782077,
  'f1': 0.5846535699197296},
 'strict': {'correct': 4931,
  'incorrect': 1131,
  'partial': 0,
  'missed': 3758,
  'spurious': 3054,
  'possible': 9820,
  'actual': 9116,
  'precision': 0.540917068889864,
  'recall': 0.5021384928716904,
  'f1': 0.5208069286016055},
 'exact': {'correct': 5009,
  'incorrect': 1053,
  'partial': 0,
  'missed': 3758,
  'spurious': 3054,
  'possible': 9820,
  'actual': 9116,
  'precision': 0.5494734532689777,
  'recall': 0.510081466395112,
  'f1': 0.5290452049007183}}

In [29]:
len(true_responses), len(pred_responses)

(2422, 4864)

In [26]:
len(sentences)

2422

In [27]:
len(generated_responses)

2422